In [62]:
import numpy as np
from scipy import sparse
import scipy.sparse.linalg as spslinalg
import matplotlib.pyplot as plt
%matplotlib inline

import a_mps as mps
import b_model as model
import c_tebd as tebd
import tfi_exact

In [95]:
# L : sites in unit cell
# a)
class MPS(mps.MPS):
    def get_theta2(self, i):
        """Calculate effective two-site wave function on sites i,j=(i+1) in mixed canonical form.

        The returned array has legs ``vL, i, j, vR``."""
        j = (i + 1) % self.L
        return np.tensordot(self.get_theta1(i), self.Bs[j], [2, 0])  # vL i [vR], [vL] j vR

    def get_chi(self):
        """Return bond dimensions."""
        return [self.Bs[i].shape[2] for i in range(self.L)]

    def bond_expectation_value(self, op):
        """Calculate expectation values of a local operator at each bond."""
        result = []
        for i in range(self.L):
            theta = self.get_theta2(i)  # vL i j vR
            op_theta = np.tensordot(op[i], theta, axes=[[2, 3], [1, 2]])
            # i j [i*] [j*], vL [i] [j] vR
            result.append(np.tensordot(theta.conj(), op_theta, [[0, 1, 2, 3], [2, 0, 1, 3]]))
            # [vL*] [i*] [j*] [vR*], [i] [j] [vL] [vR]
        return np.real_if_close(result)

    def entanglement_entropy(self):
        """Return the (von-Neumann) entanglement entropy for a bipartition at any of the bonds."""
        result = []
        for i in range(1, self.L+1):
            S = self.Ss[i].copy()
            S[S < 1.e-20] = 0.  # 0*log(0) should give 0; avoid warning or NaN.
            S2 = S * S
            assert abs(np.linalg.norm(S) - 1.) < 1.e-14
            result.append(-np.sum(S2 * np.log(S2)))
        return np.array(result)

In [66]:
class TFIModel(model.TFIModel):
    def __init__(self, L, J, g):
        super().__init__(L, J, g)
    
    def init_H_bonds(self):
        """Initialize `H_bonds` hamiltonian. Called by __init__()."""
        sx, sz, id = self.sigmax, self.sigmaz, self.id
        d = self.d
        H_list = []
        for i in range(self.L):
            gL = gR = 0.5 * self.g
            # if i == 0: # first bond
            #     gL = self.g
            # if i + 1 == self.L: # last bond
            #     gR = self.g
            H_bond = -self.J * np.kron(sx, sx) - gL * np.kron(sz, id) - gR * np.kron(id, sz)
            # H_bond has legs ``i, j, i*, j*``
            H_list.append(np.reshape(H_bond, [d, d, d, d]))
        self.H_bonds = H_list
    
    def energy(self, psi):
        """Evaluate energy density (per site) E = <psi|H|psi> / L for the given MPS."""
        assert psi.L == self.L
        return np.sum(psi.bond_expectation_value(self.H_bonds)) / psi.L

In [67]:
def update_bond(psi, i, U_bond, chi_max, eps):
    """Apply `U_bond` acting on i,j=(i+1) to `psi`."""
    j = (i + 1) % psi.L
    # construct theta matrix
    theta = psi.get_theta2(i)  # vL i j vR
    # apply U
    Utheta = np.tensordot(U_bond, theta, axes=([2, 3], [1, 2]))  # i j [i*] [j*], vL [i] [j] vR
    Utheta = np.transpose(Utheta, [2, 0, 1, 3])  # vL i j vR
    # split and truncate
    Ai, Sj, Bj = tebd.split_truncate_theta(Utheta, chi_max, eps)
    # put back into MPS
    Gi = np.tensordot(np.diag(psi.Ss[i]**(-1)), Ai, axes=[1, 0])  # vL [vL*], [vL] i vC
    psi.Bs[i] = np.tensordot(Gi, np.diag(Sj), axes=[2, 0])  # vL i [vC], [vC] vC
    psi.Ss[j] = Sj  # vC
    psi.Bs[j] = Bj  # vC j vR

In [5]:
# b)
def run_TEBD(psi, U_bonds, N_steps, chi_max, eps):
    """Evolve the state `psi` for `N_steps` time steps with (first order) TEBD.

    The state psi is modified in place."""
    Nbonds = psi.L
    assert len(U_bonds) == Nbonds
    for n in range(N_steps):
        for k in [0, 1]:  # even, odd
            for i_bond in range(k, Nbonds, 2):
                update_bond(psi, i_bond, U_bonds[i_bond], chi_max, eps)

In [12]:
def init_spinup_MPS(L):
    """Return a product state with all spins up as an MPS"""
    B = np.zeros([1, 2, 1], np.float)
    B[0, 0, 0] = 1.
    S = np.ones([1], np.float)
    Bs = [B.copy() for i in range(L)]
    Ss = [S.copy() for i in range(L)]
    return MPS(Bs, Ss)

In [117]:
def TEBD_gs_infinite(L, J, g, dts=[0.1], N_steps=500, chi_max=30, eps=1.e-10):
    print("Infinite TEBD (imaginary time evolution)")
    print("L={L:d}, J={J:.1f}, g={g:.2f}".format(L=L, J=J, g=g))
    
    tising = TFIModel(L, J, g)
    psi = init_spinup_MPS(L)
    
    for dt in dts:
        U_bonds = tebd.calc_U_bonds(tising, dt)
        run_TEBD(psi, U_bonds, N_steps, chi_max, eps)
        E = tising.energy(psi)
        print("dt = {dt:.5f}: \t\t E_dens = {E:.13f}".format(dt=dt, E=E))
    print("Final bond dimensions:  ", psi.get_chi())
    
    if L < 10:  # for small cell size compare to exact diagonalization
        E_exact = tfi_exact.infinite_gs_energy(J, g)
        print("Exact diagonalization:   E_dens = {E:.13f}".format(E=E_exact))
        print("Relative error: \t", abs((E - E_exact) / E_exact))
    
    return E, psi, tising

In [113]:
# c)
def contract_TransferMatrix(psi):
    L = psi.L
    chi = psi.Bs[0].shape[0]
    Up = psi.Bs[0]
    Low = psi.Bs[0].conj()

    for i in range(1, L):
        Up = np.tensordot(Up, psi.Bs[i], axes=1) # vL [vR] [vL] vR
        Low = np.tensordot(Low, psi.Bs[i].conj(), axes=1) # vL [vR] [vL] vR
    T = np.tensordot(Up, Low, axes=(range(1, L+1), range(1, L+1))) # vL [i] ... [j] vR
    T = T.transpose((0, 2, 1, 3)).reshape((chi**2, chi**2))
    return T

In [114]:
mps_spinup = init_spinup_MPS(L=3)
contract_TransferMatrix(mps_spinup)

array([[1.]])

In [119]:
L = 2
J = 1.
g = 1.
E, psi, tising = TEBD_gs_infinite(L, J, g, dts=[0.1], N_steps=1000, chi_max=30, eps=1.e-10);

Infinite TEBD (imaginary time evolution)
L=2, J=1.0, g=1.00
dt = 0.10000: 		 E_dens = -1.2689531576511
Final bond dimensions:   [30, 30]
Exact diagonalization:   E_dens = -1.2732395447352
Relative error: 	 0.0033665205434278977


In [123]:
transf = contract_TransferMatrix(psi)
eigval, eigvec = spslinalg.eigs(transf, k=3, which='LM')
np.abs(eigval)

array([1.00092046, 0.99069237, 0.97504241])